# TalkingData AdTracking Fraud Detection Challenge

## Introduction

_TalkingData AdTracking Fraud Detection Challenge_ is an online competition hosted by [TalkingData](https://www.talkingdata.com/) on [Kaggle](https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection), a platform for predictive modelling and analytics. In this competition, we need to identify fraudulent clicks on online Ads.

On the vast landscape of the Internet, fraud risk is everywhere. For the online advertising industry, fraudulent clicks on Ads can happen at an overwhelming volume, resulting invalid Ads coverage and tremendous money waste for advertisers.

The goal of this competition is to build an algorithm that predicts real Ad clicks (i.e. A user will download an app after clicking a mobile app Ad.). A dataset covering approximately 200 million clicks over 4 days is provided.

## Data

In [1]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt
%matplotlib inline
import time
import gc

Before loading the dataset, we can define the data types to save memory. This is a common trick when working with very large dataset.

In [2]:
start_time = time.time()

dtypes = {'ip': 'uint32',
          'app': 'uint16',
          'device': 'uint16',
          'os': 'uint16',
          'channel': 'uint16',
          'is_attributed': 'uint8',
          'click_id': 'uint32'}

Due to the size of the entire dataset, I loaded only a subset to work on within the limit of memory.

In [3]:
%%time
skiprows = np.sort(np.random.choice(np.arange(1, 184903891), size=184903891-1-80000000, replace=False)) # -1 excludes the header

print('# of skiped data points:', 184903891-1-80000000)

# of skiped data points: 104903890
CPU times: user 28.4 s, sys: 1.93 s, total: 30.3 s
Wall time: 28.5 s


In [4]:
# %%time
# # testing using a small subset of data
# skiprows = np.sort(np.random.choice(np.arange(1, 184903891), size=184903891-1-50000000, replace=False)) # -1 excludes the header

# print('# of skiped data points:', 184903891-1-50000000)

To process dataset more efficiently, we can leverage Amazon Web Services (AWS)'s Elastic Compute (EC2) resources. I chose the instance type m5.4xlarge which came with 16 CPUs and 64 (GiB) Memory.

In [5]:
%%time
train_data = pd.read_csv('/home/ec2-user/.kaggle/competitions/talkingdata-adtracking-fraud-detection/train.csv',
                         usecols=['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed'],
                         skiprows=skiprows,
                         dtype=dtypes,
                         parse_dates=['click_time'],
                         infer_datetime_format=True, engine='c')

label = train_data['is_attributed']
del train_data['is_attributed']
del skiprows
gc.collect()

CPU times: user 2min 23s, sys: 7.14 s, total: 2min 30s
Wall time: 3min 5s


In [6]:
%%time
test_data = pd.read_csv('/home/ec2-user/.kaggle/competitions/talkingdata-adtracking-fraud-detection/test.csv',
                        usecols=['ip', 'app', 'device', 'os', 'channel', 'click_time', 'click_id'],
                        dtype=dtypes,
                        parse_dates = ['click_time'],
                        infer_datetime_format=True, engine='c')

test_data['click_id'].to_csv('click_id.csv', index=False)
del test_data['click_id']
gc.collect()

CPU times: user 47.1 s, sys: 1.26 s, total: 48.3 s
Wall time: 46 s


In [7]:
train_data_len = train_data.shape[0]

print('train_data size:', train_data.shape)
print('test_data size:', test_data.shape)

train_data size: (80000000, 6)
test_data size: (18790469, 6)


In [8]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000000 entries, 0 to 79999999
Data columns (total 6 columns):
ip            uint32
app           uint16
device        uint16
os            uint16
channel       uint16
click_time    datetime64[ns]
dtypes: datetime64[ns](1), uint16(4), uint32(1)
memory usage: 1.5 GB


In [9]:
train_data.head(5)

,ip,app,device,os,channel,click_time
0,83230,3,1,13,379,2017-11-06 14:32:21
1,17357,3,1,19,379,2017-11-06 14:33:34
2,35810,3,1,13,379,2017-11-06 14:34:12
3,18787,3,1,16,379,2017-11-06 14:36:26
4,165970,3,1,13,379,2017-11-06 14:38:10


Since the dataset is highly unbalanced, it is important that we measure the unbalanced ratio for further use in classification model training.

In [10]:
is_attributed_value_counts = label.value_counts()
attributed_ratio = is_attributed_value_counts[1]/is_attributed_value_counts[0]

print('attributed:', is_attributed_value_counts[0])
print('not attributed:', is_attributed_value_counts[1])
print('attributed ratio: {0:.6f}'.format(attributed_ratio))
print('positive label weight benchmark: {0:.6f}'.format(1/attributed_ratio))

attributed: 79802443
not attributed: 197557
attributed ratio: 0.002476
positive label weight benchmark: 403.946421


In [11]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18790469 entries, 0 to 18790468
Data columns (total 6 columns):
ip            uint32
app           uint16
device        uint16
os            uint16
channel       uint16
click_time    datetime64[ns]
dtypes: datetime64[ns](1), uint16(4), uint32(1)
memory usage: 358.4 MB


In [12]:
test_data.head(5)

,ip,app,device,os,channel,click_time
0,5744,9,1,3,107,2017-11-10 04:00:00
1,119901,9,1,3,466,2017-11-10 04:00:00
2,72287,21,1,19,128,2017-11-10 04:00:00
3,78477,15,1,13,111,2017-11-10 04:00:00
4,123080,12,1,13,328,2017-11-10 04:00:00


In [13]:
%%time
full_data = pd.concat([train_data, test_data])

del train_data
del test_data
gc.collect()

full_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98790469 entries, 0 to 18790468
Data columns (total 6 columns):
ip            uint32
app           uint16
device        uint16
os            uint16
channel       uint16
click_time    datetime64[ns]
dtypes: datetime64[ns](1), uint16(4), uint32(1)
memory usage: 2.6 GB
CPU times: user 2.32 s, sys: 1.69 s, total: 4.01 s
Wall time: 968 ms


## Feature Engineering

In [14]:
%%time
full_data['minute'] = pd.to_datetime(full_data.click_time).dt.minute.astype('uint8')
full_data['hour'] = pd.to_datetime(full_data.click_time).dt.hour.astype('uint8')
# full_data['day'] = pd.to_datetime(full_data.click_time).dt.day.astype('uint8')
# full_data['month'] = pd.to_datetime(full_data.click_time).dt.month.astype('uint8')
full_data['dow']  = pd.to_datetime(full_data.click_time).dt.dayofweek.astype('uint8')
# full_data['doy']  = pd.to_datetime(full_data.click_time).dt.dayofyear.astype('uint32')

CPU times: user 15 s, sys: 5.45 s, total: 20.5 s
Wall time: 12.2 s


In [15]:
%%time
print('minute set:', set(full_data['minute']))
print('hour set:', set(full_data['hour']))
# print('day set:', set(full_data['day']))
# print('month set:', set(full_data['month']))
print('dow set:', set(full_data['dow']))
# print('doy set:', set(full_data['doy']))

minute set: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59}
hour set: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}
dow set: {0, 1, 2, 3, 4}
CPU times: user 6.03 s, sys: 900 ms, total: 6.93 s
Wall time: 5.61 s


In [16]:
# %%time
# # divide hour range to 4 sections
# full_data['hour_1'] = full_data['hour'].apply(lambda x: 1 if x >= 0 and x <= 6  else 0).astype('uint8')
# full_data['hour_2'] = full_data['hour'].apply(lambda x: 1 if x > 6 and x <= 12 else 0).astype('uint8')
# full_data['hour_3'] = full_data['hour'].apply(lambda x: 1 if x > 12 and x <= 18 else 0).astype('uint8')
# full_data['hour_4'] = full_data['hour'].apply(lambda x: 1 if x > 18 and x <= 23 else 0).astype('uint8')

In [17]:
%%time
# # divide day range to 2 sections
full_data['hour_working'] = full_data['hour'].apply(lambda x: 1 if x >= 9 and x <= 18 else 0).astype('uint8')
# full_data['hour_off_working'] = full_data['hour_working'].apply(lambda x: 0 if x == 1 else 1).astype('uint8')

CPU times: user 23.7 s, sys: 2.3 s, total: 26 s
Wall time: 25.9 s


In [18]:
full_data.dtypes.sort_index()

app                     uint16
channel                 uint16
click_time      datetime64[ns]
device                  uint16
dow                      uint8
hour                     uint8
hour_working             uint8
ip                      uint32
minute                   uint8
os                      uint16
dtype: object

In [19]:
# referneces: 
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.unique.html
# https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.ravel_multi_index.html

# add counts of selected columns
def col_counts(df, cols):
    np_arr = df[cols].values
    unique, unique_inverse, unique_counts = np.unique(np.ravel_multi_index(np_arr.T, np_arr.max(0) + 1), \
                                         return_inverse=True, return_counts=True)
    
    new_col = "_".join(cols)+'_unicount'
    df[new_col] = unique_counts[unique_inverse]
    
    # use appropriate data type
    max_value = df[new_col].max()
    if max_value > 65536:
        df[new_col] = df[new_col].astype('uint32')
    else:
        df[new_col] = df[new_col].astype('uint16')

In [20]:
# refernce: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html
# add frequencies of selected columns
def col_freq(df, col, cols):
    new_col = col + 'freq_' + "_".join(cols)
    tmp = df[cols + [col]].groupby(by=cols)[col].count().reset_index().rename(index=str, columns={col: new_col})
    
    # use appropriate data type
    max_value = tmp[new_col].max()
    if max_value > 65536:
        tmp[new_col] = tmp[new_col].astype('uint32')
    else:
        tmp[new_col] = tmp[new_col].astype('uint16')
    
    return tmp

In [21]:
%%time
# unique counts of ip and app combination
col_counts(full_data, ['ip', 'app'])

# unique counts of ip, app, and channel combination
col_counts(full_data, ['ip', 'app', 'channel'])

# unique counts of ip, day, and hour combination
col_counts(full_data, ['ip', 'app', 'os'])

# unique counts of ip and channel combination
col_counts(full_data, ['ip', 'channel'])

# unique counts of ip and device combination
col_counts(full_data, ['ip', 'device'])

# unique counts of app and channel combination
col_counts(full_data, ['app', 'channel'])
gc.collect()

CPU times: user 4min 23s, sys: 52.2 s, total: 5min 15s
Wall time: 3min 42s


In [22]:
%%time
# count number of day-of-week-occurances per ip
full_data = full_data.merge(col_freq(full_data, 'dow', ['ip']), on=['ip'], how='left', copy=False)

# count number of apps per ip
full_data = full_data.merge(col_freq(full_data, 'app', ['ip']), on=['ip'], how='left', copy=False)

# count number of app per day of week per channel clicks per ip
full_data = full_data.merge(col_freq(full_data, 'app', ['ip', 'dow', 'channel']), \
                            on=['ip', 'dow', 'channel'], how='left', copy=False)
gc.collect()

CPU times: user 4min 37s, sys: 4min 41s, total: 9min 19s
Wall time: 3min 38s


In [23]:
%%time
# count number of channel clicks per ip
full_data = full_data.merge(col_freq(full_data, 'channel', ['ip']), on=['ip'], how='left', copy=False)

# count number of channel clicks per ip per app
full_data = full_data.merge(col_freq(full_data, 'channel', ['ip', 'app']), on=['ip', 'app'], how='left', copy=False)

# count number of channel clicks per ip per device
full_data = full_data.merge(col_freq(full_data, 'channel', ['ip', 'device']), on=['ip', 'device'], how='left', copy=False)
gc.collect()

# count number of channel clicks per ip per day of week per app
full_data = full_data.merge(col_freq(full_data, 'channel', ['ip', 'device', 'app']), \
                            on=['ip', 'device', 'app'], how='left', copy=False)

# count number of channel clicks per ip per device per os
full_data = full_data.merge(col_freq(full_data, 'channel', ['ip', 'device', 'os']), \
                            on=['ip', 'device', 'os'], how='left', copy=False)
gc.collect()

CPU times: user 9min 16s, sys: 12min 38s, total: 21min 55s
Wall time: 7min 23s


In [24]:
%%time
# count number of channel clicks per ip per day of week
full_data = full_data.merge(col_freq(full_data, 'channel', ['ip', 'dow']), on=['ip', 'dow'], how='left', copy=False)

# count number of channel clicks per ip per day of week per app
full_data = full_data.merge(col_freq(full_data, 'channel', ['ip', 'dow', 'app']), \
                            on=['ip', 'dow', 'app'], how='left', copy=False)
gc.collect()

# count number of channel clicks per ip per day of week per device
full_data = full_data.merge(col_freq(full_data, 'channel', ['ip', 'dow', 'device']), \
                            on=['ip', 'dow', 'device'], how='left', copy=False)

# count number of channel clicks per ip per day of week per hour
full_data = full_data.merge(col_freq(full_data, 'channel', ['ip', 'dow', 'hour']), \
                            on=['ip', 'dow', 'hour'], how='left', copy=False)
gc.collect()

CPU times: user 7min 56s, sys: 14min 13s, total: 22min 10s
Wall time: 6min 27s


In [25]:
%%time
# count number of channel clicks per ip per day of week per hour per minute
full_data = full_data.merge(col_freq(full_data, 'channel', ['ip', 'dow', 'hour', 'minute']), \
                            on=['ip', 'dow', 'hour', 'minute'], how='left', copy=False)

# count number of channel clicks per ip per day of week per hour per os per app
full_data = full_data.merge(col_freq(full_data, 'channel', ['ip', 'dow', 'hour', 'os', 'app']), \
                            on=['ip', 'dow', 'hour', 'os', 'app'], how='left', copy=False)
gc.collect()

# count number of channel clicks per ip per day of week per os
full_data = full_data.merge(col_freq(full_data, 'channel', ['ip', 'dow', 'os']), \
                            on=['ip', 'dow', 'os'], how='left', copy=False)

# count number of channel clicks per ip per day of week per os per app
full_data = full_data.merge(col_freq(full_data, 'channel', ['ip', 'dow', 'os', 'app']), \
                            on=['ip', 'dow', 'os', 'app'], how='left', copy=False)
gc.collect()

CPU times: user 12min 14s, sys: 19min 27s, total: 31min 41s
Wall time: 10min 32s


In [26]:
%%time
# count number of hours between the earliset and latest click times
tmp = full_data[['ip', 'click_time']].groupby(by=['ip'])['click_time'].max().reset_index()\
.rename(index=str, columns={'click_time': 'click_time_max'})

full_data = full_data.merge(tmp, on=['ip'], how='left')
del tmp
gc.collect()

tmp2 = full_data[['ip', 'click_time']].groupby(by=['ip'])['click_time'].min().reset_index()\
.rename(index=str, columns={'click_time': 'click_time_min'})

full_data = full_data.merge(tmp2, on=['ip'], how='left')
del tmp2
gc.collect()

full_data['duration'] = full_data['click_time_max'] - full_data['click_time_min']
del full_data['click_time_max']
del full_data['click_time_min']

full_data['duration_in_hours'] = full_data['duration'].apply(lambda x: x.seconds/60).round().astype('uint16')
del full_data['duration']
del full_data['click_time']
gc.collect()

CPU times: user 15min 37s, sys: 9min 43s, total: 25min 21s
Wall time: 14min 46s


In [27]:
%%time
# count number of channel clicks per ip per hour per app
full_data = full_data.merge(col_freq(full_data, 'channel', ['ip', 'hour', 'app']), \
                            on=['ip', 'hour', 'app'], how='left', copy=False)

# count number of channel clicks per ip per hour per device
full_data = full_data.merge(col_freq(full_data, 'channel', ['ip', 'hour', 'device']), \
                            on=['ip', 'hour', 'device'], how='left', copy=False)
gc.collect()

# count number of channel clicks per ip per hour per os
full_data = full_data.merge(col_freq(full_data, 'channel', ['ip', 'hour', 'os']), \
                            on=['ip', 'hour', 'os'], how='left', copy=False)

# count number of channel clicks per ip per os
full_data = full_data.merge(col_freq(full_data, 'channel', ['ip', 'os']), \
                            on=['ip', 'os'], how='left', copy=False)
gc.collect()

# count number of channel clicks per ip per os per app
full_data = full_data.merge(col_freq(full_data, 'channel', ['ip', 'os', 'app']), \
                            on=['ip', 'os', 'app'], how='left', copy=False)

CPU times: user 14min 6s, sys: 29min 21s, total: 43min 27s
Wall time: 11min 32s


In [28]:
%%time
# count number of devices per ip
full_data = full_data.merge(col_freq(full_data, 'device', ['ip']), on=['ip'], how='left', copy=False)
gc.collect()

# count number of devices per app per ip
full_data = full_data.merge(col_freq(full_data, 'device', ['ip', 'app']), on=['ip', 'app'], how='left', copy=False)

# count number of devices per day of week per channel clicks per ip
full_data = full_data.merge(col_freq(full_data, 'device', ['ip', 'dow', 'channel']), \
                            on=['ip', 'dow', 'channel'], how='left', copy=False)
gc.collect()

# count number of devices per hour per ip
full_data = full_data.merge(col_freq(full_data, 'device', ['ip', 'hour']), on=['ip', 'hour'], how='left', copy=False)

# count number of devices per channel clicks per ip
full_data = full_data.merge(col_freq(full_data, 'device', ['ip', 'os', 'channel']), \
                            on=['ip', 'os', 'channel'], how='left', copy=False)
gc.collect()

CPU times: user 17min 49s, sys: 43min 50s, total: 1h 1min 40s
Wall time: 14min 40s


In [29]:
%%time
# count number of os per ip
full_data = full_data.merge(col_freq(full_data, 'os', ['ip']), on=['ip'], how='left', copy=False)
gc.collect()

# count number of os per channel clicks per ip
full_data = full_data.merge(col_freq(full_data, 'os', ['ip', 'channel']), on=['ip', 'channel'], how='left', copy=False)
gc.collect()

# count number of os per hour per channel clicks per ip
full_data = full_data.merge(col_freq(full_data, 'os', ['ip', 'hour', 'channel']), \
                            on=['ip', 'hour', 'channel'], how='left', copy=False)
gc.collect()

CPU times: user 12min 58s, sys: 34min 37s, total: 47min 36s
Wall time: 10min 52s


In [30]:
print(full_data.shape)
print(full_data.dtypes.sort_index())

(98790469, 45)
app                               uint16
app_channel_unicount              uint32
appfreq_ip                        uint32
appfreq_ip_dow_channel            uint16
channel                           uint16
channelfreq_ip                    uint32
channelfreq_ip_app                uint32
channelfreq_ip_device             uint32
channelfreq_ip_device_app         uint32
channelfreq_ip_device_os          uint32
channelfreq_ip_dow                uint32
channelfreq_ip_dow_app            uint16
channelfreq_ip_dow_device         uint32
channelfreq_ip_dow_hour           uint16
channelfreq_ip_dow_hour_minute    uint16
channelfreq_ip_dow_hour_os_app    uint16
channelfreq_ip_dow_os             uint16
channelfreq_ip_dow_os_app         uint16
channelfreq_ip_hour_app           uint16
channelfreq_ip_hour_device        uint32
channelfreq_ip_hour_os            uint16
channelfreq_ip_os                 uint32
channelfreq_ip_os_app             uint16
device                            uint16
d

In [31]:
full_data.head(10)

,ip,app,device,os,channel,minute,hour,dow,hour_working,ip_app_unicount,...,channelfreq_ip_os,channelfreq_ip_os_app,devicefreq_ip,devicefreq_ip_app,devicefreq_ip_dow_channel,devicefreq_ip_hour,devicefreq_ip_os_channel,osfreq_ip,osfreq_ip_channel,osfreq_ip_hour_channel
0,83230,3,1,13,379,32,14,0,1,2642,...,3490,675,13460,2642,12,617,55,13460,210,10
1,17357,3,1,19,379,33,14,0,1,2385,...,3450,663,12601,2385,16,636,65,12601,218,12
2,35810,3,1,13,379,34,14,0,1,994,...,1059,234,5424,994,3,629,13,5424,52,8
3,18787,3,1,16,379,36,14,0,1,409,...,60,18,1910,409,2,99,2,1910,34,2
4,165970,3,1,13,379,38,14,0,1,196,...,110,19,895,196,1,76,1,895,8,1
5,172522,3,1,25,379,38,14,0,1,628,...,109,16,4066,628,43,1,10,4066,106,1
6,210962,3,1,19,379,39,14,0,1,533,...,815,145,3785,533,3,289,15,3785,44,1
7,124979,3,1,18,379,40,14,0,1,77,...,26,5,471,77,2,8,1,471,10,1
8,80447,3,1,19,379,40,14,0,1,422,...,409,85,2653,422,2,100,13,2653,43,3
9,134575,3,1,13,379,43,14,0,1,114,...,185,32,1418,114,2,149,3,1418,16,1


In [ ]:
# check duplicated features
# full_data.T.drop_duplicates().T

In [33]:
features = full_data.columns.tolist()
features.sort()

In [34]:
# remove useless features
features.remove('minute')
features.remove('ip')
# features.remove('day')

In [35]:
features

['app',
 'app_channel_unicount',
 'appfreq_ip',
 'appfreq_ip_dow_channel',
 'channel',
 'channelfreq_ip',
 'channelfreq_ip_app',
 'channelfreq_ip_device',
 'channelfreq_ip_device_app',
 'channelfreq_ip_device_os',
 'channelfreq_ip_dow',
 'channelfreq_ip_dow_app',
 'channelfreq_ip_dow_device',
 'channelfreq_ip_dow_hour',
 'channelfreq_ip_dow_hour_minute',
 'channelfreq_ip_dow_hour_os_app',
 'channelfreq_ip_dow_os',
 'channelfreq_ip_dow_os_app',
 'channelfreq_ip_hour_app',
 'channelfreq_ip_hour_device',
 'channelfreq_ip_hour_os',
 'channelfreq_ip_os',
 'channelfreq_ip_os_app',
 'device',
 'devicefreq_ip',
 'devicefreq_ip_app',
 'devicefreq_ip_dow_channel',
 'devicefreq_ip_hour',
 'devicefreq_ip_os_channel',
 'dow',
 'dowfreq_ip',
 'duration_in_hours',
 'hour',
 'hour_working',
 'ip_app_channel_unicount',
 'ip_app_unicount',
 'ip_channel_unicount',
 'ip_device_unicount',
 'ip_dow_unicount',
 'os',
 'osfreq_ip',
 'osfreq_ip_channel',
 'osfreq_ip_hour_channel']

## Save Processed Data for Later Training and Validation

In [38]:
%%time
test_data_processed = full_data[features][train_data_len:]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(full_data[features][:train_data_len], label, test_size=0.1, shuffle=False)
gc.collect()

test_data_processed.to_csv('test_data_processed.csv', index=False)
X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
y_test.to_csv('y_test.csv', index=False)

# del full_data
gc.collect()

CPU times: user 53min 19s, sys: 3min 35s, total: 56min 55s
Wall time: 52min 36s
